# <span style="color:red;"> Semáforos</span>

Ya se han visto el la teoría la diferencia entre hilos y procesos. En este notebook vamos a aprender a utilizar semáforos en hilos en lugar de procesos porque simplifica la compartición de variables.

## <span style="color:orange;">Funcionalidad</span>

Un semáforo se puede ver como una variable que tiene un valor entero:
* Puede iniciarse con un valor no negativo.
* La operación down (wait) (P según la notación Dijkstra):
    * Si semaforo > 0: disminuye el valor del semáforo
    * Si semaforo = 0: el proceso se duerme.
* La operación up (signal) (V según la notación Dijkstra) incrementa el valor del semáforo.
    * Si hay procesos durmiendo en ese semáforo despierta a alguno de ellos.
    * De lo contrario incrementa el valor del semáforo.

## <span style="color:orange;">Generalizando un poco</span>

Vamos a hacer algo un poco más genérico en el que creamos un procedimiento Launcher. En el primer parámetro se almacenan los identificadores de los hilos en forma de lista. Es una función de parámetros variables en el que se pasa un conjunto de hilos a crear e iniciar. Con el fin de iniciar todos los hilos lo más simultáneamente posible se espera a que sean creados todos antes de ser lanzados.

Así mismo vamos a empezar a trabajar con semáforos. Por simplicidad para aprender el uso de semáforos vamos a utilizar los semáforos de hilos para aprender a trabajar con ellos. En el código de la siguiente celda también se dedinen los procedimientos _up_ y _down_ con el fin de que los problemas a resolver tengan un aspecto muy similar al pseudocódigo de los problemas típicos.

In [2]:
from threading import Thread, Semaphore
from time import sleep
from random import choice,randint
from string import ascii_letters
import os


def Launcher(threads,*funcs):
    for func in funcs:
        t = Thread(target=func)
        threads.append(t)
    for thr in threads:
        thr.start()
        
def down(sem):
    sem.acquire()

def up(sem):
    sem.release()
    
def SemaphoreArray(N,value=1):
    array = []
    for i in range(N):
        array.append(Semaphore(value))
    return array

Vamos a hacer una pequeña prueba de estos procedimientos. Creamos cuatro procedimientos cada uno controlado por un semáforo inicializado a 0 de tal forma que los hilos se duermen al hacer el _down_ al principio del hilo. En el procedimiento principal haremos un _up_ de cada uno de estos semáforos.

In [3]:
s1 = Semaphore(0)
s2 = Semaphore(0)
s3 = Semaphore(0)
s4 = Semaphore(0)
 
def uno():
    global s1
    down(s1)
    print("Uno")
    up(s1)
    return
        
def dos():
    global s2
    down(s2)
    print("Dos")
    up(s2)
    return
        
def tres():
    global s3
    down(s3)
    print("Tres")
    up(s3)
    return
        
def cuatro():
    global s4
    down(s4)
    print("Cuatro")
    up(s4)
    return

threads = []

Launcher(threads, uno, dos, tres, cuatro)


up(s4)
up(s2)
up(s1)
sleep(5)
up(s3)

Dos
UnoCuatro

Tres


Prueba varias veces el código anterior. ¿Por qué no aparecen los mensajes en el mismo orden en el que se hacen los _up_?

## <span style="color:orange;">Problemas típicos con semáforos</span>
### <span style="color:yellow;">Problema del Productor - consumidor</span>

#### <span style="color:cyan;">Enunciado del productor-consumidor</span>

Uno o varios procesos producen un bien que guardan en un almacén y uno o varios procesos consumen ese bien del almacén. El almacé tendrá un tamaño limitado.

#### <span style="color:cyan;">Implementación del productor-consumidor</span>

Vamos a crear un productor y un consumidor con un almacén de tamaño limitado (N). Para que estos hilos no sean infinitos se sustituye el bucle infinito por un bucle de tamaño limitado. Así mismo, para poder controlar cuándo se ejecutan los dos procesos se añade un semáforo controlador para el productor y otro para el consumidor.

In [5]:
N=24
INTENTOS = 30
NPRODCONS = 60
almacen = ['*'] * N
elemin = 0
elemout = 0

mutex = Semaphore(1)
llenos = Semaphore(0)
vacios = Semaphore(N)
prod = Semaphore(0)
cons = Semaphore(0)

def productor():
    global almacen, elemin, elemout
    i=0
    
    while i < NPRODCONS:
        i += 1
        down(prod) #auxiliar
        down(vacios)
        down(mutex)
        almacen[elemin] = choice(ascii_letters)
        print(almacen)
        elemin = (elemin+1)%N
        up(mutex)
        up(llenos)
    return
    
def consumidor():
    global almacen, elemin, elemout
    i=0
    
    while i < NPRODCONS:
        i += 1
        down(cons) #auxiliar
        down(llenos)
        down(mutex)
        a = almacen[elemout]
        almacen[elemout] = '*'
        print(almacen)
        elemout = (elemout+1)%N
        up(mutex)
        up(vacios)
    return

trs= []
Launcher(trs,productor, consumidor)

# Controlando la ejecución con los semáforos auxiliares
for i in range(26):
    up(prod)
    
sleep(3)

for i in range(2):
    up(cons)
    
sleep(3)

for i in range(5):
    up(cons)
    
sleep(3)
    
for i in range(INTENTOS):
    if randint(0,1):
        up(cons)
    else:
        up(prod)
    sleep(1)

['I', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
['I', 'b', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
['I', 'b', 'y', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
['I', 'b', 'y', 'e', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
['I', 'b', 'y', 'e', 'h', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
['I', 'b', 'y', 'e', 'h', 's', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
['I', 'b', 'y', 'e', 'h', 's', 'Q', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
['I', 'b', 'y', 'e', 'h', 's', 'Q', 'b', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
['I', 'b', 'y', 'e', 'h', 's', '

#### <span style="color:cyan;">Cuestiones</span>

1. Intenta explicar el resultado.
2. ¿Qué sucede si se quita el último sleep? Pruébalo y explica qué sucede.
3. ¿Qué sucede si se cambia el orden relativo del mutex respecto a los semáforos de vacíos y llenos. Pruébalo.
4. ¿El código es válido para múltiples productores y consumidores? Haz las pruebas necesarias.

## <span style="color:orange;">Problema del baile de salón</span>
### <span style="color:yellow;">Enunciado del problema del baile de salón</span>
En el hotel Hastor de New York existe una sala de baile en la que los hombres se ponen en una fila y las mujeres en otra de tal forma que salen a bailar por parejas en el orden en el que están en la fila. Por supuesto ni un hombre ni una mujer pueden salir a bailar sólos ni quedarse en la pista sólos. Sin embargo no tienen por qué salir con la pareja con la que entraron.

In [6]:
m1 = Semaphore(0)
m2 = Semaphore(0)
m3 = Semaphore(0)
m4 = Semaphore(0)

def Lider(m1,m2):
    up(m1)
    down(m2)
    
def Follower(m1,m2):
    down(m1)
    up(m2)
    
def Mujer(numero):
    Lider(m1,m2)
    Baila("mujer",numero)
    Follower(m3,m4)
    Sale("mujer",numero)

def Hombre(numero):
    Follower(m1,m2)
    Baila("hombre",numero)
    Lider(m3,m4)
    Sale("hombre",numero)
    
def Baila(genero, numero):
    print("Bailo, soy %s y mi número es %d" %(genero, numero))
    sleep(randint(1,3))
    
def Sale(genero, numero):
    print("Salgo, soy %s y mi número es %d" %(genero, numero))
    sleep(randint(1,3))
    
def H0(): Hombre(0)
def H1(): Hombre(1)
def H2(): Hombre(2)
def H3(): Hombre(3)
def H4(): Hombre(4)
def H5(): Hombre(5)
def H6(): Hombre(6)
    
def M0(): Mujer(0)
def M1(): Mujer(1)
def M2(): Mujer(2)
def M3(): Mujer(3)
def M4(): Mujer(4)
def M5(): Mujer(5)
def M6(): Mujer(6)
    
trs= []
Launcher(trs,H0,H1,H2,H3,H4,H5,H6)
print("Hay hombres en la cola esperando")
sleep(5)
print("Aparecen tres mujeres en la cola")
trs= []
Launcher(trs,M0,M1,M2)
sleep(5)
print("Y ahora cuatro más mujeres en la cola")
trs= []
Launcher(trs,M3,M4,M5,M6)

Hay hombres en la cola esperando
Aparecen tres mujeres en la cola
Bailo, soy hombre y mi número es 0
Bailo, soy mujer y mi número es 0
Bailo, soy hombre y mi número es 1
Bailo, soy mujer y mi número es 1
Bailo, soy hombre y mi número es 2
Bailo, soy mujer y mi número es 2
Salgo, soy mujer y mi número es 1Salgo, soy hombre y mi número es 2

Salgo, soy mujer y mi número es 2
Salgo, soy hombre y mi número es 0
Salgo, soy mujer y mi número es 0Salgo, soy hombre y mi número es 1

Y ahora cuatro más mujeres en la cola
Bailo, soy hombre y mi número es 3
Bailo, soy mujer y mi número es 3
Bailo, soy hombre y mi número es 4
Bailo, soy mujer y mi número es 4
Bailo, soy hombre y mi número es 5
Bailo, soy mujer y mi número es 5
Bailo, soy hombre y mi número es 6
Bailo, soy mujer y mi número es 6
Salgo, soy mujer y mi número es 4Salgo, soy hombre y mi número es 6

Salgo, soy mujer y mi número es 6Salgo, soy hombre y mi número es 4

Salgo, soy mujer y mi número es 5Salgo, soy hombre y mi número es 5


#### <span style="color:cyan;">Problema</span>
Lanza distintos bailarines en el orden en el que desees y mira qué sucede. Cambia también el tiempo de baile en los distintos procesos. Por ejemplo, que a las mujeres les guste mucho más que a los hombres el quedarse bailando. ¿Qué sucede?

## <span style="color:orange;">Problema de los babuinos</span>
### <span style="color:yellow;">Enunciado del problema de los babuinos</span>
En el parque nacional Kruger en Sudafrica hay un cañón muy profundo con una simple cuerda para cruzarlo. Los babuinos necesitan cruzar ese cañón constantemente en ambas direcciones gracias a una cuerda. Sin embargo:
- Como los babuinos son muy agresivos, si dos de ellos se encuentran en cualquier punto de la cuerda yendo en direcciones opuestas, estos se pelearán y terminarán cayendo por el cañón y muriendo.
- La cuerda no es muy resistente y aguanta a un máximo de cinco babuinos simultáneamente. Si en cualquier instante hay más de cinco babuinos en la cuerda, esta se romperá y los babuinos caerán también al vacío.

#### <span style="color:cyan;">Definiendo el LightSwitch</span>
Dado que en python no es posible pasar por valor es necesario para poder mantener variables dentro del Light Switch en definirlos en forma de clase clase. En las siguientes líneas se define esa clase con dos métodos para encender y apagar la luz.

In [8]:
class lsw:
    def __init__(self,resourceSem):
        self.mutex = Semaphore(1)
        self.counter = 0
        self.resourceSem = resourceSem

    def LightSwitchOn(self,texto):
        print("QUIERE " + texto)
        down(self.mutex)
        self.counter += 1
        if self.counter == 1:
            down(self.resourceSem)
            print("APROPIADO " +texto)
        up(self.mutex)

    def LightSwitchOff(self,texto):
        down(self.mutex)
        self.counter -= 1
        print("SALE " + texto)
        if self.counter == 0:
            print("LIBERADO "+ texto)
            up(self.resourceSem)
        up(self.mutex)

En las siguientes líneas vamos a crear los procedimientos para ser lanzados como hilos. Se puede comprobar que coinciden perfectamente con el pseudocódigo presenado en las clases de teoría.

In [9]:
N = 5 # Número máximo de babuinos en la cuerda
maxBabuinosVa = Semaphore(N)
maxBabuinosViene = Semaphore(N)
cuerda = Semaphore(1)

def cruza(texto):
    print ("CRUZA " + texto)
    sleep(randint(1,3))

def BabuinoVa(clase):
    global maxBabuinosVa,cuerda
    down(maxBabuinosVa)
    clase.LightSwitchOn("VA\n")
    cruza("VA\n")
    clase.LightSwitchOff("VA\n")
    up(maxBabuinosVa)
    
def BabuinoViene(clase): 
    global maxBabuinosViene,cuerda
    down(maxBabuinosViene)
    clase.LightSwitchOn("VIENE\n")
    cruza("VIENE\n") 
    clase.LightSwitchOff("VIENE\n")
    up(maxBabuinosViene) 

Ya sólo nos falta probarlo. Para ello creamos hilos con babuinos que quieren ir o venir aleatoriamente. Creamos 20 de esos hilos.

In [10]:
va = lsw(cuerda)
viene = lsw(cuerda)

for i in range(20):
    if randint(0,1):
        t=Thread(target=BabuinoVa,args=(va,))
    else:
        t=Thread(target=BabuinoViene,args=(viene,))
    t.start()

QUIERE VIENE

APROPIADO VIENE

CRUZA VIENE

QUIERE VA

QUIERE VA

QUIERE VIENE

CRUZA VIENE

QUIERE VIENE

CRUZA VIENE

QUIERE VA

QUIERE VIENE

CRUZA VIENE

QUIERE VIENE

CRUZA VIENE

QUIERE VA

QUIERE VA

SALE VIENE

QUIERE VIENE

CRUZA VIENE

SALE VIENE

QUIERE VIENE

CRUZA VIENE

SALE VIENE

QUIERE VIENE

CRUZA VIENE

SALE VIENE

QUIERE VIENE

CRUZA VIENE

SALE VIENE

SALE VIENE
QUIERE VIENE


QUIERE VIENE

CRUZA VIENE

CRUZA VIENE

SALE VIENE

SALE VIENE

SALE VIENE

SALE VIENE

SALE VIENE

LIBERADO VIENE

APROPIADO VA

CRUZA VA

CRUZA VA

CRUZA VA
CRUZA VA

CRUZA VA


SALE VA

QUIERE VA

CRUZA VA

SALE VA

SALE VA
QUIERE VA


CRUZA VA

QUIERE VA

CRUZA VA

SALE VA

SALE VA
QUIERE VA


CRUZA VA

SALE VA

SALE VA

SALE VA

SALE VA

LIBERADO VA



Como se puede ver, esta solución no previene inanición.

#### <span style="color:cyan;">Cuestiones y problemas</span>
1. Comprueba a partir de la salida que se ha resuelto correctamente el problema.
2. Intenta modificar el resultado para que no haya inanición.

## <span style="color:orange;">Problema de la cena de los filósofos</span>
El problema de la cena de los filósofos (dining philosophers problem) es un problema clásico de las ciencias de la computación propuesto por Edsger Dijkstra en 1965 para representar el problema de la sincronización de procesos en un sistema operativo. Cabe aclarar que la interpretación está basada en pensadores chinos, quienes comían con dos palillos, donde es más lógico que se necesite el del comensal que se siente al lado para poder comer.

### <span style="color:yellow;">Enunciado del problema</span>

Cinco filósofos se sientan alrededor de una mesa y pasan su vida cenando y pensando. Cada filósofo tiene un plato de fideos y un tenedor a la izquierda de su plato. Para comer los fideos son necesarios dos tenedores y cada filósofo sólo puede tomar los que están a su izquierda y derecha. Si cualquier filósofo toma un tenedor y el otro está ocupado, se quedará esperando, con el tenedor en la mano, hasta que pueda tomar el otro tenedor, para luego empezar a comer.

Si dos filósofos adyacentes intentan tomar el mismo tenedor a una vez, se produce una condición de carrera: ambos compiten por tomar el mismo tenedor, y uno de ellos se queda sin comer.

Si todos los filósofos toman el tenedor que está a su derecha al mismo tiempo, entonces todos se quedarán esperando eternamente, porque alguien debe liberar el tenedor que les falta. Nadie lo hará porque todos se encuentran en la misma situación (esperando que alguno deje sus tenedores). Entonces los filósofos se morirán de hambre. Este bloqueo mutuo se denomina interbloqueo o _deadlock_.

El problema consiste en encontrar un algoritmo que permita que los filósofos no se mueran de hambre.

### <span style="color:yellow;">Solución</span>
A continuación se presenta una solución para el problema en el que no se previene inanición de algún filósofo.

In [3]:
N = 5 # Número de filósofos
INTENTOS = 5

def izq(nfilosofo):
    return (nfilosofo -1) % N

def der(nfilosofo):
    return (nfilosofo +1) % N

estado = ["PENSANDO"]*N
semFilosofo = [0]*N
mutex = Semaphore(1)

def pensar(max,nfilosofo):
    print("PENSANDO %d\n" % nfilosofo)
    sleep(randint(0,max))
    
def comer(max,nfilosofo):
    print("COMIENDO %d\n" % nfilosofo)
    sleep(randint(0,max))
    
def intenta(nfilosofo):
    if estado[nfilosofo] == "HAMBRIENTO" and estado[izq(nfilosofo)] != "COMIENDO" and estado[der(nfilosofo)] != "COMIENDO":
        estado[nfilosofo] = "COMIENDO"
        up(semFilosofo[nfilosofo])
        
def cogeTenedores(nfilosofo):
    down(mutex)
    estado[nfilosofo]="HAMBRIENTO"
    intenta(nfilosofo)
    up(mutex)
    down(semFilosofo[nfilosofo])
    
def dejaTenedores(nfilosofo):
    down(mutex)
    estado[nfilosofo]="PENSANDO"
    intenta(izq(nfilosofo))
    intenta(der(nfilosofo))
    up(mutex)
    
def filosofo(nfilosofo):
    for j in range(INTENTOS):
        pensar(N,nfilosofo)
        cogeTenedores(nfilosofo)
        comer(N,nfilosofo)
        dejaTenedores(nfilosofo)
    
t = []
for i in range(N):
    semFilosofo[i] = Semaphore(0)
    t= Thread(target=filosofo,args=(i,))
    t.start()
    

PENSANDO 0
PENSANDO 1


PENSANDO 2

PENSANDO 3
PENSANDO 4


COMIENDO 3

COMIENDO 0

PENSANDO 0

PENSANDO 3
COMIENDO 2
COMIENDO 4



PENSANDO 2
COMIENDO 1


PENSANDO 4
COMIENDO 3


PENSANDO 3

PENSANDO 1

COMIENDO 2

COMIENDO 0

PENSANDO 0
COMIENDO 4


PENSANDO 2

COMIENDO 1

PENSANDO 1

COMIENDO 1

PENSANDO 4
COMIENDO 3


PENSANDO 1

COMIENDO 0

PENSANDO 3
COMIENDO 2


PENSANDO 0
COMIENDO 4


PENSANDO 2
COMIENDO 1


PENSANDO 4
COMIENDO 3


PENSANDO 1

COMIENDO 0

PENSANDO 0

PENSANDO 3
COMIENDO 4


COMIENDO 2

PENSANDO 4

COMIENDO 0

PENSANDO 2

COMIENDO 3

COMIENDO 2

COMIENDO 1
COMIENDO 4




#### <span style="color:cyan;">Cuestiones</span>
1. Identifica qué hacen los procedimientos __izq__ y __der__
2. El procedimiento más importante es el procedimiento __intenta__. Identifica su funcionamiento e identifica por qué cuando un filósofo deja los tenedores hace que lo intenten los filósofos que están a su izquierda y a su derecha.

## <span style="color:orange;">Otros problemas</span>
En las siguientes celdas se van a enunciar un conjunto de problemas típicos para que pruebes a solucionarlos con las herramientas que ya dispones. Todos ellos han sido extraidos del libro ["El pequeño libro de los semáforos"](https://greenteapress.com/wp/semaphores/).

Para aprender bien se aconseja que añadas una o varias celdas después de cada problema y que sigas el siguiente procedimiento para resolverlos.
1. Analiza el problema en papel.
    1. Identifica los procesos que deben existir.
    2. Identifica los puntos de concurrencia y ordénalos en cada proceso.
    3. Encuentra una forma de solucionar cada punto de concurrencia.
    4. Añade los _mutex_ necesarios para proteger la memoria compartida para que no se produzcan condiciones de carrera.
    5. Analiza el código resultante y comprueba que no hay problemas de interbloqueos.
    6. Analiza el problema y comprueba si hay problemas de inanición y si estos son resolubles sin desaprovechar recursos.
2. Impleméntalo usando los recursos disponibles en este "_notebook_".
3. Añade las salidas necesarias para comprobar que funciona correctamente.
4. Si algo no funciona vuelve al punto 1.

### <span style="color:yellow;">Problema de los lectores-escritores</span>
Este problema se refiere a cualquier situación en la que una estructura de datos, base de datos o sistema de archivos es leído y modificado por hilos concurrentes. Mientras se está escribiendo o modificando la estructura de datos, a menudo es necesario impedir la lectura de otros hilos, para evitar que un lector interrumpa una modificación en curso y lea de forma inconsistente, o bien datos no válidos. Como en el problema productor-consumidor, la solución es asimétrica. Los lectores y escritores ejecutan códigos diferentes antes de entrar en la sección crítica. Las restricciones de sincronización son:
1. Cualquier número de lectores puede estar en la sección crítica simultáneamente.
2. Los escritores deben tener acceso exclusivo a la sección crítica. En otras palabras, un escritor no puede entrar en la sección crítica mientras que cualquier otro hilo (lector o escritor) está allí, y mientras el escritor está allí, ningún otro hilo puede entrar.

In [4]:
cuenta_lectores = 0

mutex = Semaphore(1)
recurso = Semaphore(1)

def leyendo():
    print("\033[32mLeyendo\033[00m")
    sleep(randint(1,10))
    print("\033[32mDejando de leer\033[00m")
    
def escribiendo():
    print("\033[31mEscribiendo\033[00m")
    sleep(randint(1,10))
    print("\033[31mDejando de escribir\033[00m")
    
def lector():
    global cuenta_lectores
    
    down(mutex)
    cuenta_lectores += 1
    if cuenta_lectores == 1:
        down(recurso)
    up(mutex)
    
    leyendo()
    
    down(mutex)
    cuenta_lectores -= 1
    if cuenta_lectores == 0:
        up(recurso)
    up(mutex)
    
def escritor():
    down(recurso)
    escribiendo()
    up(recurso)
    
th1 = []
th2 = []
th3 = []
th4 = []
th5 = []

Launcher(th1,lector,lector,lector, escritor, lector)
sleep(9)
Launcher(th2,escritor,lector,lector, lector, lector)
sleep(9)
Launcher(th3,lector,lector,lector, escritor, lector)
sleep(9)
Launcher(th4,lector,lector,escritor, lector, lector)
sleep(9)
Launcher(th5,lector,lector,lector, lector, escritor)
    

Leyendo
Leyendo
Leyendo
Leyendo
Dejando de leer
Dejando de leer
Dejando de leer
Dejando de leer
Escribiendo
Dejando de escribir
Escribiendo
Dejando de escribir
Leyendo
Leyendo
Leyendo
LeyendoLeyendo
Leyendo

Leyendo
Leyendo
Dejando de leer
Leyendo
Leyendo
Dejando de leer
LeyendoLeyendo

Dejando de leer
Dejando de leer
Dejando de leer
Dejando de leer
Dejando de leer
Dejando de leer
Dejando de leer
Dejando de leer
Dejando de leer
Dejando de leer
Escribiendo
Dejando de escribir
Escribiendo
Dejando de escribir
Leyendo
Leyendo
Leyendo
Leyendo
Dejando de leer
Dejando de leer
Dejando de leer
Dejando de leer
Escribiendo
Dejando de escribir


### <span style="color:yellow;">Problema de los lectores-escritores con prioridad a escritores</span>
Es el mismo problema que el anterior pero si un escritor desea escribir no puede leer ningún lector nuevo hasta que no se realice la o las escrituras.

### <span style="color:yellow;">Problema de la cena de los salvajes</span>
Una tribu de salvajes realiza cenas comunales de una olla grande que puede contener M porciones de misioneros guisados. Cuando un salvaje quiere comer, se sirve de la olla, a menos que esté vacía. Si la olla está vacía, el salvaje despierta al cocinero que cocina a un nuevo misionero y luego espera hasta que el cocinero haya vuelto a llenar la olla. Evidentemente si otro quiere cenar, la olla está vacía pero el cocinero está despierto no tiene que despertarlo pero esperará a que llene la olla.

**Variante**: Se pueden concinar sólo hasta N misioneros que fueron el total de misioneros capturados en la última cacería de misioneros.

In [ ]:
sem_olla = Semaphore(1)
olla_vacia = Semaphore(0)
olla_llena = Semaphore(0)

def savage():
    down(sem_olla)
    if porciones == 0:
        up(olla_vacia)
        down(olla_llena)
    porciones -= 1
    up(sem_olla)
    como_misionero()
    
def cocinero():
    while True:
        down(olla_vacia)
        llena_olla()
        up(olla_llena)

### <span style="color:yellow;">Problema del H<sub>2</sub>O</span>
Este problema ha sido un elemento básico de la clase de sistemas operativos en la Universidad de California en Berkeley durante al menos una década. 

Hay dos tipos de átomos: oxígeno e hidrógeno, y queremos crear moléculas de agua. Para ello tenemos un catalizador que espera a que estén disponibles dos átomos de hidrógeno y uno de oxígeno para porducir una molécula de agua.
Para ensamblar estos átomos en moléculas de agua tenemos que crear una barrera que haga que cada átomo espere a que estén disponibles el resto  espere hasta que una molécula completa esté lista para proceder. A medida que cada hilo pasa la barrera, debe invocar la unión. Debes garantizar que todos los hilos de una molécula invoquen la unión antes de que lo haga cualquiera de los hilos de la siguiente molécula. Como los átomos son indistinguibles basta con que haya dos hidrógenos y un oxígeno para que puedan reaccionar.

In [5]:
N=2

mutex = Semaphore(1)
oxigenos = Semaphore(0)
hidrogenos = Semaphore(0)
contador = 0

def junta():
    print("\n\033[36mAgua!!!!\033[00m\n")
    
def catalizador():
    for i in range(N):
        down(oxigenos)
        down(hidrogenos)
        junta()
        
def oxigeno():
    print("Oxigeno")
    up(oxigenos)
    
def hidrogeno():
    global contador
    print("Hidrogeno")
    down(mutex)
    contador += 1
    if contador %2 == 0:
        up(hidrogenos)
        contador -= 2
    up(mutex)
    

t1 = []
t2 =[]
t3 = []
t4 =[]

Launcher(t1,catalizador)
Launcher(t2,oxigeno,oxigeno)
sleep(5)
Launcher(t3,hidrogeno, hidrogeno,hidrogeno)
sleep(5)
Launcher(t4,hidrogeno)

Oxigeno
Oxigeno
Hidrogeno
Hidrogeno

Agua!!!!
Hidrogeno

Hidrogeno

Agua!!!!



### <span style="color:yellow;">Problema de la montaña rusa</span>
Supongamos que hay n de pasajeros y un coche. Los pasajeros esperan repetidamente para dar una vuelta en la montaña rusa, que puede contener C pasajeros. El coche de la montaña rusa espera a estar lleno para empezar el viaje.

Aquí hay algunos detalles adicionales:
- Los pasajeros deben invocar el _embarque_ y _desembarque_.
- El coche debe invocar la _carga_, _corre_ y _descarga_.
- Los pasajeros no pueden embarcar hasta que el coche haya invocado la carga.
- El coche no puede salir hasta que se haya llenado.
- Los pasajeros no pueden desembarcar hasta que el coche no haya invocado la descarga.

In [ ]:
ontador_pasajeros_S = 0
contador_pasajeros_B = 0
N = 5
vueltas = 7

mutexS = Semaphore(1)
mutexB = Semaphore(1)
cargar = Semaphore(0)
cargados = Semaphore(0)
descargar = Semaphore(0)
descargados = Semaphore(0)

def embarque():
    global contador_pasajeros_S
    down(mutexS)
    contador_pasajeros_S += 1
    if contador_pasajeros_S == 1:
        down(cargar)
        print("Empezando a embarcar")
    if contador_pasajeros_S == N:
        contador_pasajeros_S = 0
        print("Embarcando el último")
        up(cargados)
    up(mutexS)

def carga():
    up(cargar)
    print("Cargando")
    down(cargados)
    
def desembarque():
    global contador_pasajeros_B
    down(mutexB)
    contador_pasajeros_B += 1
    if contador_pasajeros_B == 1:
        down(descargar)
        print("Empezando a desembarcar")
    if contador_pasajeros_B == N:
        contador_pasajeros_B = 0
        print("Desembarcando el último")
        up(descargados)
    up(mutexB)
    
def descarga():
    up(descargar)
    print("Descargando")
    down(descargados)

def corre():
    print("\033[31mCorriendo\033[00m")
    sleep(5)
    
def pasajero():
    embarque()
    desembarque()
    
def coche():
    print("\033[32mEmpiezan los viajes\033[00m")
    for i in range(vueltas):
        carga()
        corre()
        descarga()
    print("\033[32mSe acabaron los viajes\033[00m")
    
    
t1 = []
t2 =[]
t3 = []
t4 =[]

Launcher(t1,coche)
Launcher(t2,pasajero,pasajero,pasajero,pasajero)
sleep(5)
Launcher(t3,pasajero,pasajero,pasajero)
sleep(5)
Launcher(t4,pasajero,pasajero,pasajero)

pasajeros = [pasajero]*25

t5 =[]
Launcher(t5,*pasajeros)

### <span style="color:yellow;">Problema del bar de sushi</span>
Imagina un bar de sushi con 5 asientos. Si llega un cliente y hay un asiento vacío, puede tomar asiento inmediatamente. Pero si cuando llega cuando los 5 asientos están llenos, eso significa que pueden están cenando juntos, y tendrá que esperar a que todos se vayan antes de sentarse.

In [ ]:
asientos_ocupados = 0
N = 5

mutex = Semaphore(1)
mutex2 = Semaphore(1)
sentarse = Semaphore(1)

def me_siento():
    print("\033[31mSentandome\033[00m")
    sleep(randint(1,10))
    print("\033[32mLevantándome\033[00m")
    
def cliente():
    global N, asientos_ocupados
    
    down(mutex2)
    down(sentarse)
    up(sentarse)
    down(mutex)
    asientos_ocupados += 1
    if asientos_ocupados == N:
        down(sentarse)
    up(mutex)
    up(mutex2)
    
    me_siento()
    
    down(mutex)
    asientos_ocupados -= 1
    if asientos_ocupados == 0:
        up(sentarse)
    up(mutex)

th1 = []
Launcher(th1,cliente,cliente,cliente)
sleep(5)

th2 = []
Launcher(th2,cliente,cliente,cliente,cliente,cliente,cliente)


### <span style="color:yellow;">La tarta de cumpleaños</span>

En un cumpleaños de ninos pequeños se pretende repartir trozos de tarta a todos los niños. Como un día
especial que es, los niños pueden repetir tantas veces como quieran y cuando quieran pero usando un sistema de rondas, de tal forma que hasta que todos los niños no han recibido su trozo de tarta de una ronda, los niños que ya han tomado tarta no pueden repetir.

Implementa el proceso **ninoPideTarta(int NumNino)** teniendo en cuenta que los semáforos que se van a utilizar son semáforos blandos (tambien denominados debiles). Se dispone de la funcion auxiliar **comeTarta()** para representar la tarea de comer el trozo de tarta. De igual forma, se utilizarán las funciones **up(semaforo)** y **down(semaforo)** para manipular los semáforos. Se pueden crear otras funciones si se consideran necesarias.

In [ ]:
from random import shuffle

N = 10
maximo = 5

n0 = Semaphore(1)
n1 = Semaphore(1)
n2 = Semaphore(1)
n3 = Semaphore(1)
n4 = Semaphore(1)
n5 = Semaphore(1)
n6 = Semaphore(1)
n7 = Semaphore(1)
n8 = Semaphore(1)
n9 = Semaphore(1)
ninnos = SemaphoreArray(N,value=1)
mutex = Semaphore(1)

cuenta = 0

def tarta(i):
    print("\033[32mEl niño %d ha pedido un trozo de tarta\033[00m" % i)

def comeTarta(i):
    print("\033[36mEl niño %d está comiéndose la tarta\033[00m" % i)
    sleep(randint(1,10))
    print("\033[36mEl niño %d se ha acabado la tarta\033[00m" % i)

def ninoPideTarta(i):
    global N, cuenta, ninnos
    orden = [0,1,2,3,4,5,6,7,8,9] # Para simular el semáforo blando se despiertan según este orden
    for c in range(maximo): # Les dejamos comer un número máximo de veces
        down(ninnos[i]) # La vez siguiente se dormirá si no han comido tarta todos los niños
                        # Se le da un trozo de tarta
        tarta(i)
        down(mutex)
        cuenta += 1 # Cuenta los niños
        if cuenta == N: # Si todos los niños han pedido tarta ...
            cuenta = 0
            shuffle(ninnos) # Se cambia aleatoriamente el orden para simular blandos
            for i in range(N): # Se les deja volver a pedir a todos
                up(ninnos[i]) 
        up(mutex)
        comeTarta(i)
        
def P0(): ninoPideTarta(0)
def P1(): ninoPideTarta(1)
def P2(): ninoPideTarta(2)
def P3(): ninoPideTarta(3)
def P4(): ninoPideTarta(4)
def P5(): ninoPideTarta(5)
def P6(): ninoPideTarta(6)
def P7(): ninoPideTarta(7)
def P8(): ninoPideTarta(8)
def P9(): ninoPideTarta(9)

th = []

Launcher(th,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9)

### <span style="color:yellow;">Problema de Santa Klaus</span>

Santa Klaus es un hombre muy mayor y necesita dormir mucho; por ello aprovecha siempre que puede para
dormir.

Los elfos saben de su necesidad y han decidido que aunque haya un problema en la fábrica de juguetes no
le despertarán, pero cuando haya tres problemas entonces sí que lo haran.

La fábrica y la oficina de Santa estan muy alejadas, por lo que si un elfo va en busca de Santa no vuelve hasta que Santa resuelva su problema.

Los elfos irán a despertar a Santa pero si no hay otros dos elfos esperando es que no hay suficientes problemas como para despertar a Santa, por ello y ya que no tienen nada que hacer se quedarán a la espera. El tercer elfo hará lo mismo pero despertara a Santa. Cuando Santa resuelva los problemas en la fábrica los tres elfos regresarán a la fábrica.

Así mismo los renos estan de vacaciones todo el año y vuelven por navidad para tirar del trineo de Santa. Cuando llegue el ultimo de los 9 renos despertará a Santa para irse a repartir los juguetes.

No es necesario controlar lo que sucede con los elfos y la produccion cuando Santa esta de reparto.

In [ ]:
NE = 3
NR = 9

mutex = Semaphore(1)
mutexElfo = Semaphore(1)
mutexReno = Semaphore(1)
elfo = Semaphore(0)
reno = Semaphore(0)
santa = Semaphore(0)
estado =[]
nrenos = 0
nelfos = 0

def resuelveProblemas():
    print("\033[31mResolviendo problemas\033[00m")
    sleep(randint(1,5))
    print("\033[31mResueltos\033[00m")
    
def reparteRegalos():
    print("\033[32mRepartiendo regalos\033[00m")
    sleep(randint(30,60))
    print("\033[32mRepartidos\033[00m")

def produce():
    sleep(randint(1,50))

def vacaciones():
    sleep(randint(30,40))
    
    
def elfoTrabaja():
    global estado,NE,nelfos
    for i in range(5):
        produce()
        down(mutexElfo)
        print("Problema elfo")
        nelfos += 1
        if nelfos >= NE:
            nelfos -= NE
            down(mutex)
            estado.append("ELFOS")
            up(mutex)
            up(santa)
        up(mutexElfo)
        down(elfo)
    
def renoViene():
    global estado,NR,nrenos
    vacaciones()
    down(mutexReno)
    print("Llega reno")
    nrenos += 1
    if nrenos == NR:
        nrenos = 0
        down(mutex)
        estado.append("RENOS")
        up(mutex)
        up(santa)
    up(mutexReno)
    down(reno)
    
def santaKlaus():
    global estado, NR, NE
    for a in range(36):
        down(santa)
        down(mutex)
        if "RENOS" in estado:
            ind = estado.index("RENOS")
            estado = estado[0:ind]+estado[ind+1:]
            up(mutex)
            for i in range(NR): up(reno)
            reparteRegalos()
        elif estado[0] == "ELFOS":
            estado = estado[1:]
            up(mutex)
            for i in range(NE): up(elfo)
            resuelveProblemas()
        
threads = []
for i in range(21):
    t = Thread(target=elfoTrabaja)
    threads.append(t)
for i in range(9):
    t = Thread(target=renoViene)
    threads.append(t)
t = Thread(target=santaKlaus)
threads.append(t)
for thr in threads:
    thr.start()

### <span style="color:yellow;">El baño unisex de la EHS</span>

En la Escuela de Hacking Superior todos los aseos son unisex. Para usar estos aseos hay un selector a la entrada que tiene tres posiciones: hombres, mujeres y libre. El cartel de hombres lo pone el primer hombre que entre, el de mujeres la primera mujer que entre y el de libre lo pone cualquiera al salir siempre que quede vacío el aseo. Cuando esta puesto el cartel de mujeres sólo pueden entrar mujeres y cuando pone el cartel de hombres sólo pueden entrar hombres. En el aseo sólo caben 10 personas por lo que cuando está lleno se activara un cartel que indica que está lleno y los que lleguen esperarán fuera a que haya sitio.

Resuelve  el  problema  implementando  el  pseudocodigo  de  los  procesos **mujer()** y de **hombre()** para que cumplan las condiciones. No es necesario prevenir la inanición.

In [ ]:
N=10
maximoH = Semaphore(N)
maximoM = Semaphore(N)
mutexM = Semaphore(1)
mutexH = Semaphore(1)

aseo = Semaphore(1)

hombresC = 0
mujeresC = 0

def usandoAseoM():
    print("Usando (\033[31mM\033[00m)")
    sleep(randint(1,3))
    print("Saliendo (\033[31mM\033[00m)")
    
def usandoAseoH():
    print("Usando (\033[34mH\033[00m)")
    sleep(randint(1,3))
    print("Saliendo (\033[34mH\033[00m)")
    
def M():
    global mujeresC
    sleep(randint(1,40))
    down(maximoM)
    down(mutexM)
    mujeresC += 1
    if mujeresC == 1:
        down(aseo)
        print("\033[31mMujeres\033[00m")
    up(mutexM)
    
    usandoAseoM()
    
    down(mutexM)
    mujeresC -= 1
    if mujeresC == 0:
        print("\033[32mLibre\033[00m")
        up(aseo)
    up(mutexM)
    up(maximoM)
    
def H():
    global hombresC
    sleep(randint(1,40))
    
    down(maximoH)
    down(mutexH)
    hombresC += 1
    if hombresC == 1:
        down(aseo)
        print("\033[34mHombres\033[00m")
    up(mutexH)
    
    usandoAseoH()
    
    down(mutexH)
    hombresC -= 1
    if hombresC == 0:
        print("\033[32mLibre\033[00m")
        up(aseo)
    up(mutexH)
    up(maximoH)
    
th1 = []
th2 = []

Launcher(th1,H,H,H,H,H,H,H,H,H,H,H,H,H)
Launcher(th2,M,M,M,M,M,H,M,M,M,M,M,M,M,M,M,M,M,M,M,M)

### <span style="color:yellow;">Cruce automático</span>

En el ano 2056 se realiza una modificación en el código de la circulación en la que cambia la regulación de los cruces con bajo tráfico. Dada la capacidad de las comunicaciones inalambricas cuando un coche se acerca a un cruce  reserva los elementos necesarios del cruce para poder realizar la maniobra.

Si no puede acceder a una de las zonas necesarias para realizar la maniobra no accederá al cruce. Es muy importante que si las maniobras de varios coches no interfieren todos los coches puedan acceder al cruce sin ningún problema.

Crea el proceso **coche(origen,destino)** donde origen y destino determinan las zonas a utilizar. En el esquema adjunto se puede ver la numeración de zonas así como la numeración de origen/destino. Por ejemplo, si el origen es 2 y el destino es 1 es necesario reservar las zonas Z3, Z4, Z1, sin embargo de 1 a 2 hace falta reservar solo Z2. 

Utiliza las primitivas **up(sem)**, **down(sem)**, **up(sem,n)** y **down(sem,n)** (las dos últimas para hacer ups y downs multiples si es necesario). No es necesario prevenir inanición pero sí evitar interbloqueos.

In [ ]:
Z = SemaphoreArray(4,1)
mutex = Semaphore(1)

def cruza():
    sleep(randint(1,2))
    

def coche(id,origen,destino):
    if destino < origen:
        destino += 4
    down(mutex)
    for i in range(origen,destino): 
        down(Z[i%4])
        print("\033[32mOcupada Z%d\033[00m" % (i%4+1))
    up(mutex)
    
    dest = destino
    if dest > 4: dest = destino - 4
    print("Coche %d cruza de %d a %d" % (id,origen,dest))
    cruza()
    
    print("Crucé %d" % id)
    for i in range(origen,destino):
        up(Z[i%4])
        print("\033[32mLibre Z%d\033[00m" % (i%4+1))

threads = []
for i in range(20):
    orig = randint(1,4)
    dest = randint(1,4)
    while dest == orig: dest = randint(1,4)
    t = Thread(target=coche,args=(i,orig,dest))
    threads.append(t)
for thr in threads:
    thr.start()

### <span style="color:yellow;">Cadena de montaje</span>
En una fábrica de coches hay 5 cadenas de montaje. De cada una de ellas salen los coches de uno en uno. Estos coches son almacenados en una zona de espera en la que caben 10 coches. En esta zona de espera hay 3 dársenas donde paran los camiones de transporte que pueden transportar cada uno 6 coches. Los camiones nunca parten antes de estar completamente cargados. Cuando un camion desea cargar puede hacerlo siempre y cuando uno que estuviese cargando pueda terminar la carga.

Implementa los procesos **cadenaMontaje** y **cargaCamion**.

**No es necesario** controlar las posiciones donde se encuentran los coches a cargar ni implementar el acceso a las darsenas ni prevenir inanición.

In [ ]:
C = 5
D = 3
N = 10
P = 20
S = 6

mutex = Semaphore(1)
mutex2 = Semaphore(1)
mutex3 = Semaphore(1)
colas = Semaphore(1)
llenos = Semaphore(0)
vacios = Semaphore(N)
darsenas = Semaphore(D)

llenosC = SemaphoreArray(3,0)
vaciosC = SemaphoreArray(3,S)

almacen=['-']*N
cola = [['-']*S]*D
darsenasC = ["LIBRE"]*D
introduccion = 0
salida = 0

def cadenaMontaje(i):
    global introduccion,almacen
    for i in range(12):
        sleep(randint(2,4))
        down(vacios)
        down(mutex)
        almacen[introduccion] = 'c'
        introduccion = (introduccion + 1) % N
        print("Almacén: ",almacen)
        up(mutex)
        up(llenos)
    
def marcarCoches(i):
    global salida,cola,almacen
    down(colas)
    
    for pos in range(S):
        down(llenos)
        down(mutex)
        valor = almacen[salida]
        almacen[salida] ='-'
        salida = (salida + 1) % N
        up(mutex)
        up(vacios)

        down(vaciosC[i])
        down(mutex2)
        cola[i][pos] = valor
        up(mutex2)
        up(llenosC[i])
    
    up(colas)
    print("Almacén: ",almacen)
    
def embarcando(dars):
    print("\033[31mEmbarcando dársena %d\033[00m" % dars)
    sleep(randint(1,3))
    print("\033[32mEmbarcado dársena %d\033[00m" % dars)
    
def cargaCamion():
    down(darsenas)
    
    down(mutex3)
    darsn = darsenasC.index("LIBRE")
    darsenasC[darsn] = "OCUPADO"
    print("Camión en dársena %d" % darsn)
    up(mutex3)
    marcarCoches(darsn)
    
    for pos in range(S):
        down(llenosC[darsn])
        down(mutex2)
        valor = cola[darsn][pos]
        up(mutex2)
        up(vaciosC[darsn])
        embarcando(darsn)
    
    darsenasC[darsn] = "LIBRE"
    up(darsenas)
    
threads = []
for i in range(C):
    t = Thread(target=cadenaMontaje,args=(i,))
    threads.append(t)
for thr in threads:
    thr.start()
    
sleep(3)
camiones = [cargaCamion]*10
th = []
Launcher(th,*camiones)

### <span style="color:yellow;">Visita a la Sagrada Familia y museo</span>

La basílica de la Sagrada Familia en Barcelona dispone de un museo en el que se explican los métodos de construcción y maquetas de prueba de carga. los responsables de la visita de turistas a la basílica están hartos de que los turistas entren a ver la basílica sin saber lo que están viendo y han decidido que es necesario visitar el museo antes de entrar en la basílica. Además, han decidido que la basílica solo se pueda visitar con un guía que explique lo que se está viendo. Nos piden implementar un sistema que permita realizar esto de forma eficiente y con las siguientes condiciones:
* El número maximo de personas en el museo es de 100 personas.
* Cuando un visitante abandona el museo se pone en la cola de espera para que los recoja un guía.
* Hay 5 guías.
* Un guía recoge a grupos de 10 personas.
* Si no hay 10 personas en la cola de espera el guía espera a que las haya y si hay más solo coge a 10

In [ ]:
NM = 100
NG = 10
G = 5
cuenta = 0

visita = Semaphore(NM)
mutex = Semaphore(1)
guia = Semaphore(0)
grupo = Semaphore(0)
basilica = Semaphore(0)

def llega():
    sleep(randint(1,100))
    print("Llega visitante")
    
def visita_museo():
    print("\033[34mVisitando museo\033[00m")
    sleep(randint(5,8))
    print("\033[44mVisitado museo\033[00m")
    
def visita_basilica():
    print("\033[32mVisitando basilica\033[00m")
    sleep(randint(20,30))
    print("\033[42mVisitada basilica\033[00m")
    up(basilica)

def guia_basilica():
    print("\033[31mInicia guía\033[00m")
    
def termina_guia():
    global NG
    for i in range(NG): down(basilica)
    print("\033[41mFinaliza guía\033[00m")

def Visitante():
    global NG, cuenta
    llega()
    down(visita)
    visita_museo()
    up(visita)
    
    down(mutex)
    cuenta += 1
    if cuenta >= NG:
        cuenta -= NG
        up(guia)
    up(mutex)
    
    down(grupo)
    visita_basilica()
    
    
def Guia():
    global NG
    
    for g in range(5):
        down(guia)
        guia_basilica()
        for i in range(NG): up(grupo)
        termina_guia()
    
procesos = [Visitante]*200 + [Guia]*G
th=[]
Launcher(th, *procesos)

### <span style="color:yellow;">El aeropuerto de Umea</span>

Umea es una pequeña ciudad sueca en el círculo polar ártico. Debido a las inclemencias climáticas su principal medio de transporte es un pequeño avión con un máximo de 32 plazas para viajeros. El pequeño aeropuerto tiene una única terminal de embarque. Con el fin de obtener rentabilidad en la línea de comunicación con Estocolmo y reducir costes, la línea aérea se usa de forma muy similar a la de una línea de autobuses pero por seguridad con unas pocas reglas que es necesario cumplir:
1.  Cuando el avión está preparado, éste se llena con los pasajeros que estén esperando. Si hay más de 32 pasajeros esperando se llenará con 32 sin un orden determinado. Los pasajeros que no hayan podido embarcar esperarán al siguiente avión.
2.  Si no hubiera pasajeros esperando, el avión volverá de forma inmediata a Estocolmo.
3.  Si los pasajeros están embarcando nadie que llegue después de que se inicie el embarque podrá entrar en la zona de embarque y mucho menos embarcar (independientemente de si los pasajeros llenan el avión o no) y sólo podrán entrar en la zona de embarque tras la partida del avión.

In [ ]:
embarcando = Semaphore(1)
embarcado = Semaphore(0)
embarque = Semaphore(0)
mutex = Semaphore(1)

cuenta = 0

def avion_sale(total):
    print("\033[32mAvión saliendo con %d pasajeros\033[00m" % total)

def pasajero():
    global cuenta
    sleep(randint(1,20))
    down(embarcando)
    up(embarcando)
    down(mutex)
    cuenta += 1
    print("Ha llegado un pasajero y hay %d" % cuenta)
    up(mutex)
    down(embarque)
    print("Embarcando")
    up(embarcado)
    
def avion():
    global cuenta
    down(embarcando)
    down(mutex)
    if cuenta < 32: 
        ups = cuenta
        cuenta = 0
    else: 
        ups = 32
        cuenta -= 32
    up(mutex)
    for i in range(ups):
        up(embarque)
    for i in range(ups):
        down(embarcado)
    up(embarcando)
    avion_sale(ups)
    
pasajeros = [pasajero]*200

th = []
Launcher(th,*pasajeros)
    
for a in range(15):
    sleep(randint(1,3))
    avion()

### <span style="color:yellow;">Los pasos de cebra de Navalaoveja</span>

En el ayuntamiento de Navalaoveja se han gastado todo el presupuesto en mármol. Como no tienen presupuesto suficiente han despedido a policías municipales pero aún así tienen que mantener la seguridad en los pasos de cebra a la salida de los colegios. Como no hay presupuesto suficiente nos piden que implementemos un sistema basado en semáforos para que tanto los peatones como los automóviles cumplan con las condiciones siguientes sin necesidad de policías:


* Si no hay automóvil que quieran acceder al paso de cebra los peatones deben poder cruzar cuando lo deseen.
* Si no hay peatones que quieran acceder al paso de cebra los automóviles deben poder pasar cuando lo deseen.
* Si hay diez automóviles esperando a que pasen peatones deben dejar de cruzar peatones y pasar los 10 coches. Aunque haya más coches, sólo pasarán 10 si hay peatones esperando.
* Salvo en el caso anterior los peatones siempre tienen prioridad para cruzar.
* Como el uso del paso de cebra en ambas direcciones tanto de peatones como de automóviles no es exclusivo, no se tendrá en cuenta la dirección en la que desean pasar.
* Evidentemente los peatones pueden pasar en grupos pero los coches sólo pueden pasar de uno en uno. 

    Implementa los procesos **peaton** y **coche()** usando semáforos. Para ello puedes usar las funciones up(sem), down(sem). También puedes usar las funciones llegapeaton(), llegacoche(), cruzapeaton(), cruzacoche(), continuapeaton() y continuacoche().

In [ ]:
cuenta_lectores = 0

mutex = Semaphore(1)
recurso = Semaphore(1)

def cruzapeaton():
    print("\033[32mCruzando peatón\033[00m")
    sleep(randint(1,3))
    print("\033[32mPeatón ha cruzado\033[00m")
    
def cruzacoche():
    print("\033[31mCruzando coche\033[00m")
    sleep(randint(1,3))
    print("\033[31mCoche ha cruzado\033[00m")
    
def peaton():
    global cuenta_lectores
    
    down(mutex)
    print("Llega peatón")
    cuenta_lectores += 1
    if cuenta_lectores == 1:
        down(recurso)
    up(mutex)
    
    cruzapeaton()
    
    down(mutex)
    cuenta_lectores -= 1
    print("Sale peatón")
    if cuenta_lectores == 0:
        up(recurso)
    up(mutex)
    
def coche():
    llega_coche()
    print("Llega coche")
    down(recurso)
    cruzacoche()
    up(recurso)
    print("Sale coche")
    sale_coche()
    
    
c1 = [coche,coche,peaton,coche,coche,peaton,coche,coche,coche,peaton,coche,coche]
th=[]
Launcher(th,*c1)